# OceanParcels
## Run Model

In [2]:
import xarray as xr
import pandas as pd
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
import numpy as np
import datetime
from operator import attrgetter
from matplotlib import pyplot as plt

In [5]:
df = xr.open_mfdataset('../../../data/Hycom/*.nc',decode_times=False)# all of our HYCOM data
dfGrid = pd.read_pickle('processingFiles/dfS1Grid.pkl') # the grid from the first large scale survey which we will use as seeding points

In [6]:
# If a particle goes "out of bounds", delete it instead of stopping the code
def DeleteParticle(particle, fieldset, time):
    particle.delete()
    

# Create a particle that can measure the distance it has gone instead of just counting lats/lons
class DistParticle(JITParticle):  # Define a new particle class that contains three extra variables
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    
def TotalDistance(particle, fieldset, time):
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat
    
    
def myround(x, base=3):
    return base * round(x/base)

Edit the box below for the parameters (depth, filename, run days, interval) and run.

In [9]:
outputFileNamePrefix = 'processingFiles/60Day3Hour30MeterAll'  # Set the name for the files
lons = dfGrid.lonC.values+360   # Set the lon for grid
lats = dfGrid.latC.values               # Set the lat for grid
mDepth = np.copy(lons)
mDepth[:] = 30                              # Set the depth to model

filenames = {'U': df,
             'V': df}
variables = {'U': 'water_u',
             'V': 'water_v'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time',
             'depth': 'depth'}

fieldset = FieldSet.from_xarray_dataset(df, variables, dimensions,allow_time_extrapolation=True)
pset = ParticleSet(fieldset=fieldset, pclass=DistParticle,lon=lons,lat=lats, depth = mDepth)
k_dist = pset.Kernel(TotalDistance)
pset.execute(AdvectionRK4 + k_dist,
             runtime=datetime.timedelta(days=60),   # SET DAYS
             dt=datetime.timedelta(hours=3),        # SET HOURS
             output_file=pset.ParticleFile(name=outputFileNamePrefix+".nc", 
             outputdt=datetime.timedelta(hours=3)),recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled DistParticleAdvectionRK4TotalDistance ==> C:\Users\Robert\AppData\Local\Temp\parcels-tmp\d6cfef8ecf230b30a7185de82076706f_0.dll
INFO: Temporary output files are stored in out-SCBTHKVN.
INFO: You can use "parcels_convert_npydir_to_netcdf out-SCBTHKVN" to convert these to a NetCDF file during the run.
100% (5184000.0 of 5184000.0) |##########| Elapsed Time: 1:25:58 Time:  1:25:58
